In [40]:
import pymysql
import pandas as pd
from textblob import TextBlob

In [30]:
conn = pymysql.connect(host='localhost', user='root', password='vaibhav@666', database='marketingproject')
conn

In [31]:
pd.read_sql_query("show databases",conn)

C:\Users\vaibhav gupta\AppData\Local\Temp\ipykernel_18684\850058667.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("show databases",conn)


,Database
0,analtics
1,analytics
2,bank_loan
3,cxdb
4,db1
5,information_schema
6,ma_anal
7,marketingproject
8,ms_analyt
9,mysql


In [34]:
 #Create a cursor object
cursor = conn.cursor()

# Execute the query
cursor.execute("SELECT * FROM customer_reviews;")

# Fetch all results
rows = cursor.fetchall()

# Get column names from cursor description
columns = [col[0] for col in cursor.description]

# Convert to DataFrame
customer_reviews_df= pd.DataFrame(rows, columns=columns)

# Print DataFrame
customer_reviews_df.head()




,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [35]:
# Function to calculate sentiment using TextBlob
def calculate_sentiment(review):
    return TextBlob(review).sentiment.polarity  # Returns polarity between -1 and 1


In [36]:
# Function to categorize sentiment based on score and rating
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'

In [37]:
# Function to bucket sentiment scores
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

In [41]:
# Apply sentiment analysis
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)


In [42]:
customer_reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",0.103571,Mixed Positive,0.0 to 0.49
1,2,80,19,2024-12-25,5,The quality is top-notch.,1.000000,Positive,0.5 to 1.0
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.333333,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.700000,Mixed Positive,0.5 to 1.0
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",0.103571,Mixed Positive,0.0 to 0.49
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.150000,Mixed Negative,-0.49 to 0.0
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",0.103571,Mixed Negative,0.0 to 0.49
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.200000,Positive,0.0 to 0.49
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.000000,Negative,0.0 to 0.49


In [43]:
 #Save results to CSV
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment_textblob.csv', index=False)